In [25]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "aacpassword"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({cursor}))



#########################
# Dashboard Layout / View
#########################
                               
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date


app.layout = html.Div([
    html.Div('_id'='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Gage Biros: SNHU CS-340 Dashboard', html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))))),
    html.Hr(),
    dash_table.DataTable(
        
        html.Div(dcc.RadioItems(
            '_id' ='filter type',
            options=[
                {'label' : 'All Rescue Types', 'value' : 'All'},
                {'label' : 'Water Rescue', 'value':'WR'},
                {'label' : 'Wilderness Rescue', 'value':'MWR'},
                {'label' : 'Individual Tracking', 'value':'ITR'},
                {'label' : 'Reset Filter', 'value' : 'RESET'}],
            value = 'RESET',
            labelStyle={'display':'inline-block'}
        )

        ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_deletable=False,
        row_selectable="single",
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10,
        
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
        

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback(Output('datatable-id','data'),
              [Input('radio_items_id', 'value')]
             )
               
### FIX ME Add code to filter interactive data table with MongoDB queries

def update_datable(value):
        if value == 'RESET':
            df = pd.DataFrame.from_records(shelter.readAll()).to_dict('records')
            print('No Filters Selected')
        
        return df
        
        elif value == 'WR':
            df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog", "breed":{"$in":["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome":"Intact Female", "age_upon_outcome_in_weeks":{"$gte":26.0, "$lte":156.0}}))
            print('Water Rescue Dogs displayed')
        return df
        
        elif value == 'MWR':
            df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog", "breed":{"$in":["German Shepard", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}, "sex_upon_outcome":"Intact Male", "age_upon_outcome_in_weeks":{"$gte":26.0, "$lte":156.0}}))
        
        return df
        
        elif value == 'ITR':
            df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog", "breed":{"$in":["Doberman Pinscher", "Golden Retriever", "Bloodhound", "Rottweiler"]}, "sex_upon_outcome":"Intact Male", "age_upon_outcome_in_weeks":{"$gte":20.0, "$lte":300.0}}))
               
    
    
    



        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)

def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]
        
#code for pie graph

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
        dff = pd.DataFrame.from_dict(view_data)
        breeds=[]
        for i in range(0, dff.__len()-1):
            breeds.append(str(dff.iloc[i,4]))
        return[
            dcc.Graph(
                figure{
                    'data':[
                        {
                            'labels': breeds,
                            'type': 'pie'
                        }
                    ],
                    'layout':{
                        'title':'Breeds',
                    }
                })
        ]
        

#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
        dl.Marker(position=[30.75,-97.48], children=[
            dl.Tooltip(dff.iloc[0,4]),
            dl.Popup([
                html.H1("Animal Name"),
                html.P(dff.iloc[1,9])
            ])
        ])
    ])
]

    




app

SyntaxError: invalid syntax (<ipython-input-25-39bee49f551d>, line 124)